# Machine Learning Pipeline - Feature Selection

In this notebook, we pick up the transformed datasets that we saved in the previous notebook.

## Reproducibility: Setting the seed

With the aim to ensure reproducibility between runs of the same notebook, but also between the research and production environment, for each step that includes some element of randomness, it is extremely important that we **set the seed**.

In [2]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# to build the models
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# to visualise all the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [3]:
# load the train and test set with the engineered variables

# we built and saved these datasets in the previous lecture.
# If you haven't done so, go ahead and check the previous notebook
# to find out how to create these datasets

X_train = pd.read_csv('xtrain.csv')
X_test = pd.read_csv('xtest.csv')

X_train.head()

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,heatingqc,centralair,electrical,1stflrsf,2ndflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,saletype,salecondition,lotfrontage_na,masvnrarea_na,garageyrblt_na
0,0.454545,0.666667,0.388581,0.0,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.238095,0.4,1.0,1.0,0.2,0.444444,0.571429,0.492647,0.950820,0.0,0.0,0.3,0.4,0.333333,0.00000,0.333333,0.5,0.25,0.333333,0.666667,0.0,0.8,0.055280,0.0,0.0,0.190068,0.123732,1.0,0.000000,0.0,0.333333,0.309142,0.183051,0.0,0.432412,0.333333,0.0,0.333333,0.5,0.375,0.5,0.333333,0.416667,1.0,0.000000,0.0,0.25,0.626168,0.0,0.25,0.169252,0.5,0.5,0.5,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.272727,0.666667,0.25,0.0,0.0,0.0
1,0.818182,0.666667,0.429425,0.0,1.0,1.0,0.666667,0.333333,1.0,0.00,0.0,0.666667,0.4,1.0,1.0,0.6,0.666667,0.428571,0.058824,0.131148,0.0,0.0,0.9,1.0,0.333333,0.00000,0.666667,0.5,1.00,0.666667,0.666667,0.0,0.0,0.000000,0.0,0.0,0.604880,0.231260,1.0,1.000000,1.0,1.000000,0.550351,0.000000,0.0,0.514455,0.000000,0.0,0.666667,0.0,0.375,0.5,0.666667,0.333333,1.0,0.000000,0.0,0.75,0.074766,0.5,0.50,0.426657,0.5,0.5,1.0,0.000000,0.060329,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.454545,0.666667,0.75,0.0,0.0,0.0
2,1.000000,0.666667,0.558672,0.0,1.0,1.0,0.333333,0.333333,1.0,0.25,0.0,0.666667,0.4,1.0,1.0,1.0,0.666667,0.428571,0.029412,0.081967,0.0,0.0,0.9,1.0,0.333333,0.00000,0.666667,0.5,1.00,0.666667,0.666667,0.0,1.0,0.124026,0.0,0.0,0.092466,0.149918,1.0,1.000000,1.0,1.000000,0.383441,0.455690,0.0,0.607647,0.333333,0.0,0.666667,0.5,0.375,0.5,0.666667,0.500000,1.0,0.000000,0.0,1.00,0.037383,0.5,0.50,0.346968,0.5,0.5,1.0,0.198370,0.166362,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.454545,0.666667,0.75,0.0,0.0,0.0
3,0.818182,0.666667,0.445638,0.0,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.380952,0.4,1.0,1.0,0.6,0.444444,0.571429,0.323529,0.737705,0.0,0.0,0.9,1.0,0.333333,0.00000,0.333333,0.5,0.50,0.333333,0.666667,0.0,0.6,0.059887,0.4,1.0,0.051798,0.151391,1.0,1.000000,1.0,1.000000,0.385492,0.000000,0.0,0.360349,0.000000,0.5,0.333333,0.0,0.250,0.5,0.333333,0.250000,1.0,0.000000,0.0,0.25,0.411215,0.0,0.25,0.302539,0.5,0.5,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,1.0,0.0,0.545455,0.666667,0.75,0.0,0.0,0.0
4,0.818182,0.666667,0.406640,0.0,1.0,1.0,0.000000,0.333333,1.0,0.00,0.0,0.619048,0.4,1.0,1.0,0.6,0.666667,0.428571,0.088235,0.196721,0.0,0.0,0.6,0.5,0.666667,0.02375,0.333333,0.5,1.00,0.666667,1.000000,0.0,0.0,0.000000,0.0,0.0,0.529538,0.202455,1.0,0.666667,1.0,1.000000,0.500348,0.000000,0.0,0.467713,0.000000,0.0,0.666667,0.0,0.375,0.5,0.333333,0.333333,1.0,0.333333,0.6,0.75,0.112150,1.0,0.50,0.283498,0.5,0.5,1.0,0.298913,0.038391,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.454545,0.666667,0.75,0.0,0.0,0.0


In [4]:
# load the target (remember that the target is log transformed)
y_train = pd.read_csv('ytrain.csv')
y_test = pd.read_csv('ytest.csv')

y_train.head()

,saleprice
0,11.744037
1,12.139399
2,12.206073
3,11.674194
4,12.078239


In [5]:
# We will do the model fitting and feature selection
# altogether in a few lines of code

# first, we specify the Lasso Regression model, and we
# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.

# Then we use the selectFromModel object from sklearn, which
# will select automatically the features which coefficients are non-zero

# remember to set the seed, the random state in this function
sel_ = SelectFromModel(Lasso(alpha=0.001, random_state=0))

# train Lasso model and select features
sel_.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.001, random_state=0))

In [6]:
sel_.get_support().sum()

37

In [7]:
# let's visualise those features that were selected.
# (selected features marked with True)

sel_.get_support()

array([ True,  True, False, False, False, False,  True,  True, False,
        True, False,  True, False, False,  True, False,  True,  True,
       False,  True,  True, False,  True, False, False, False,  True,
       False,  True,  True, False,  True,  True, False, False, False,
       False, False, False,  True,  True, False,  True, False, False,
        True,  True, False,  True,  True, False, False,  True,  True,
        True,  True,  True, False, False,  True,  True,  True, False,
       False,  True,  True, False, False, False,  True, False, False,
       False, False, False, False,  True,  True, False, False, False])

In [10]:
# let's print the number of total and selected features

# this is how we can make a list of the selected features
selected_feats = X_train.columns[(sel_.get_support())]

# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feats)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

total features: 81
selected features: 37
features with coefficients shrank to zero: 44


In [11]:
# print the selected features
selected_feats

Index(['mssubclass', 'mszoning', 'lotshape', 'landcontour', 'lotconfig',
       'neighborhood', 'bldgtype', 'overallqual', 'overallcond',
       'yearremodadd', 'roofstyle', 'exterior1st', 'exterqual', 'foundation',
       'bsmtqual', 'bsmtexposure', 'bsmtfintype1', 'heatingqc', 'centralair',
       '1stflrsf', 'grlivarea', 'bsmtfullbath', 'fullbath', 'halfbath',
       'kitchenqual', 'totrmsabvgrd', 'functional', 'fireplaces',
       'fireplacequ', 'garagefinish', 'garagecars', 'garagearea', 'paveddrive',
       'wooddecksf', 'screenporch', 'saletype', 'salecondition'],
      dtype='object')

In [12]:
# Saving the slected features
pd.Series(selected_feats).to_csv('selected_features.csv', index=False)